In [94]:
import torch
from transformers import Trainer, TrainerCallback, pipeline
import smtplib
import subprocess
import time
import threading
import ctypes
import pandas as pd
from datasets import Dataset
    

In [95]:
import torch
torch.cuda.empty_cache()


In [96]:
import torch
print("Using GPU: ", torch.cuda.is_available())


Using GPU:  True


In [97]:

import subprocess

def keep_awake():
    subprocess.call([
        "powershell.exe",
        "-Command",
        "Add-Type -TypeDefinition @'\nusing System;\nusing System.Runtime.InteropServices;\npublic class SleepPreventer {\n    [DllImport(\"kernel32.dll\")]\n    public static extern uint SetThreadExecutionState(uint esFlags);\n}\n'@\n[SleepPreventer]::SetThreadExecutionState(0x80000002 -bor 0x00000001)"
    ])
    print("Laptop is set to stay awake during training...")


import smtplib

def send_email_notification():
    sender_email = "sadhvi.chandragiri@outlook.com"  # Replace with your Outlook email
    recipient_email = "recipient_email@example.com"  # Replace with the recipient's email
    password = "your_outlook_password"  # Replace with your Outlook email password

    subject = "Training Complete"
    body = "Your model training has completed successfully!"

    email_message = f"Subject: {subject}\n\n{body}"

    try:
        with smtplib.SMTP("smtp.office365.com", 587) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, password)  # Login to the SMTP server
            server.sendmail(sender_email, recipient_email, email_message)
            print("Email notification sent!")
    except Exception as e:
        print(f"Failed to send email: {e}")


In [98]:

# Log GPU usage
def log_gpu_usage():
    with open("gpu_usage_log.txt", "a") as f:
        while True:
            gpu_status = subprocess.check_output("nvidia-smi", shell=True).decode()
            f.write(gpu_status)
            f.write("\n" + "-"*80 + "\n")
            time.sleep(300)  # Log every 5 minutes

In [99]:
#Setup Logging Callback for Training
#Add the custom callback for logging training progress:
                       
class CustomLoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            with open("training_log.txt", "a") as f:
                f.write(f"Step {state.global_step}: {logs}\n")


In [100]:
# Define output paths
num_epochs = 3  # Replace with your actual epoch count
dataset_size = 50000

In [101]:
# Load cleaned dataset and select first 10,000 rows
data = pd.read_csv("../data/processed/recipeNLG_final.csv").head(dataset_size)
print(data.shape)

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(data)



(50000, 9)


In [102]:
from evaluate import load

# Load BLEU metric
bleu_metric = load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Prepare references as a list of lists
    decoded_labels = [[label] for label in decoded_labels]

    # Compute BLEU score
    result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


In [103]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    combined_texts = [
        ingredient + " " + direction
        for ingredient, direction in zip(examples["ingredients"], examples["directions"])
    ]
    return tokenizer(combined_texts, truncation=True, padding=True)

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [104]:
from datasets import DatasetDict

# Split the dataset into train and eval sets
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)  # 10% for evaluation
train_dataset = split_dataset["train"]
#eval_dataset = split_dataset["test"]
# Get the dataset size dynamically
#dataset_size = len(eval_dataset)
#eval_dataset = eval_dataset.select(range(min(500, dataset_size)))  # Select up to 2000 rows, or all rows if smaller





In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=f"../results/50k_epoch_{num_epochs}_{learning_rate}",  # Save directory
    eval_strategy="no",  # No validation during training
    save_strategy="epoch",  # Save the model at the end of every epoch
    learning_rate=1e-5,  # Lower learning rate for stability
    per_device_train_batch_size=2,  # Increase batch size if possible
    gradient_accumulation_steps=2,  # Effective batch size = batch_size * steps
    num_train_epochs=5,  # Increase epochs to ensure sufficient training
    save_total_limit=2,  # Keep only the 2 latest checkpoints
    logging_dir="../logs",  # Logging directory
    load_best_model_at_end=False,  # No validation, so no best model selection
    fp16=torch.cuda.is_available(),  # Enable FP16 for faster training if GPU is available
    report_to="none"  # Avoid unnecessary logs
)


In [106]:
from transformers import DataCollatorForLanguageModeling

# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to False for causal language modeling like GPT-2
)

# Update Trainer to use data_collator instead of tokenizer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    #eval_dataset=eval_dataset,  # No evaluation for Fix 1
    eval_dataset=None,
    data_collator=data_collator,  # Use data collator instead of tokenizer
    callbacks=[CustomLoggingCallback()],
)


In [107]:
import time

# Start time
start_time = time.strftime('%Y-%m-%d %H:%M:%S')

# Your script code here
time.sleep(5)  # Simulating script execution with a delay



In [108]:
# Start GPU monitoring (optional)
gpu_logger = threading.Thread(target=log_gpu_usage)
gpu_logger.start()

# Start training
keep_awake()  # Ensure the laptop doesn't sleep

# Clear GPU cache before evaluation
torch.cuda.empty_cache()

trainer.train()
torch.cuda.empty_cache()  # Free memory before evaluation
#trainer.evaluate()


Cannot convert argument "esFlags", with value: "-2147483645", for "SetThreadExecutionState" to type "System.UInt32": 
"Cannot convert value "-2147483645" to type "System.UInt32". Error: "Value was either too large or too small for a 
UInt32.""
At line:9 char:1
+ [SleepPreventer]::SetThreadExecutionState(0x80000002 -bor 0x00000001)
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : NotSpecified: (:) [], MethodException
    + FullyQualifiedErrorId : MethodArgumentConversionInvalidCastArgument
 


Laptop is set to stay awake during training...


Step,Training Loss
500,2.084300
1000,1.882400
1500,1.802900
2000,1.775900
2500,1.754200
3000,1.730100
3500,1.701600
4000,1.663800
4500,1.654400
5000,1.627200



# Notify when training completes
send_email_notification()

In [109]:
trainer.save_model(f"../model/fine_tuned_gpt2/{dataset_size}k_epoch_{num_epochs}")  # Save the model
tokenizer.save_pretrained(f"../model/fine_tuned_gpt2/{dataset_size}k_epoch_{num_epochs}")  # Save the tokenizer



('../model/fine_tuned_gpt2/50000k_epoch_3/tokenizer_config.json',
 '../model/fine_tuned_gpt2/50000k_epoch_3/special_tokens_map.json',
 '../model/fine_tuned_gpt2/50000k_epoch_3/vocab.json',
 '../model/fine_tuned_gpt2/50000k_epoch_3/merges.txt',
 '../model/fine_tuned_gpt2/50000k_epoch_3/added_tokens.json')

In [110]:

# End time
end_time = time.strftime('%Y-%m-%d %H:%M:%S')

print(f"Training started at: {start_time}")
print(f"Training ended at: {end_time}")

Training started at: 2024-12-14 23:34:28
Training ended at: 2024-12-15 02:15:01


In [111]:
import os
import itertools
import pandas as pd
from transformers import pipeline

# Define the function to save the generated recipe
def write_generated_recipes_to_csv(output_file, recipes):
    """
    Save a list of recipes to a CSV file.
    """
    df = pd.DataFrame(recipes)
    df.to_csv(output_file, index=False)
    print(f"Recipes saved to {output_file}")



In [112]:

# Define the recipe generator
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_recipes_batch(dataset, generator, max_length=200):
    """
    Generate recipes in batch for all parameter combinations.
    """
    recipes = []
    for _, row in dataset.iterrows():
        # Extract parameters and input ingredients
        input_ingredients = row["input_ingredients"]
        top_k = row["top_k"]
        top_p = row["top_p"]
        temperature = row["temperature"]

        # Generate a recipe with the current parameters
        recipe = generator(
            input_ingredients,
            max_length=max_length,
            num_return_sequences=1,
            do_sample=True,
            top_k=top_k,
            top_p=top_p,
            temperature=temperature,
        )[0]["generated_text"]

        # Append the recipe and parameters to the results
        recipes.append({
            "input_ingredients": input_ingredients,
            "top_k": top_k,
            "top_p": top_p,
            "temperature": temperature,
            "recipe": recipe
        })
    return recipes

Device set to use cuda:0


In [113]:

# Example usage
if __name__ == "__main__":
    # Input ingredients
    input_ingredients = "chicken, garlic, onion, spices"

    # Define parameter ranges
    top_k_values = [30, 50, 100]
    top_p_values = [0.8, 0.9, 0.95]
    temperature_values = [0.7, 0.8, 1.0]

    # Create all combinations of parameters
    parameter_combinations = list(itertools.product(top_k_values, top_p_values, temperature_values))

    # Prepare the dataset as a DataFrame
    dataset = pd.DataFrame(parameter_combinations, columns=["top_k", "top_p", "temperature"])
    dataset["input_ingredients"] = input_ingredients

    # Output file
    output_dir = "../results/recipes"
    os.makedirs(output_dir, exist_ok=True)
    output_file = f"{output_dir}/{dataset_size}K_{num_epochs}_batch_decoding_results.csv"

    # Generate recipes in batch
    print("Generating recipes in batch...")
    results = generate_recipes_batch(dataset, generator)

    # Save the results to a CSV file
    write_generated_recipes_to_csv(output_file, results)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generating recipes in batch...
Recipes saved to ../results/recipes/50000K_3_batch_decoding_results.csv


In [ ]:
from transformers import pipeline

# Initialize generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define parameter ranges
temperature = 0.7
top_k = 50
top_p = 0.9

# Generate recipe
input_prompt = "Chicken curry with garlic and spices, Indian cuisine, baked"
generated_recipe = generator(
    input_prompt,
    max_length=150,  # Maximum length of output
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    num_return_sequences=1  # Single output
)

print("Generated Recipe:", generated_recipe[0]["generated_text"])


input_ingredients = "Ingredients: tofu, blueberries, thyme. Instructions:"
generated_recipe = generator(
    input_ingredients,
    max_length=200,
    num_return_sequences=1,
    do_sample=True,
    top_k=40,
    top_p=0.9,
    temperature=0.8
)

print("Generated Recipe:\n", generated_recipe[0]["generated_text"])



from transformers import AutoTokenizer

# Inspect tokenized output
tokenized_sample = tokenizer("chicken, garlic, onion, spices", return_tensors="pt")
print(tokenized_sample)
print(tokenizer.convert_ids_to_tokens(tokenized_sample["input_ids"][0]))


for param in model.transformer.wte.parameters():
    param.requires_grad = False
for param in model.transformer.wpe.parameters():
    param.requires_grad = False


# Tokenize a sample of your dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Load your dataset
import pandas as pd
data = pd.read_csv("../data/processed/recipeNLG_final.csv").head(1000)  # Load a subset
sample_texts = data["text_column"]  # Replace 'text_column' with your dataset's text column

# Tokenize and analyze splits
for text in sample_texts[:10]:  # Check the first 10 samples
    tokenized_output = tokenizer(text)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_output["input_ids"])
    print(f"Text: {text}")
    print(f"Tokens: {tokens}\n")
